##          Московский авиационный институт
###      (Национальный исследовательский университет)
####   Институт №8 «Информационные технологии и прикладная математика»
####        Кафедра вычислительной математики и программирования

  
 
 





            Лабораторная работа №  
            по курсу « ».
            
           

 
 
 







                                  Студент: Макаренкова В.М.
                                  Группа: 80-406Б
                                  Преподаватель: 


 

                        Москва, 2021